Load packages

In [3]:
#loading packages
push!(LOAD_PATH, "src");
import Base: @kwdef
using Pkg
Pkg.instantiate()
using Revise
using Test
using LifeInsuranceDataModel

using BitemporalPostgres
using SearchLight
using SearchLightPostgreSQL
using TimeZones
using ToStruct
using JSON

purging the data model entirely - empty the schema

In [4]:
if (haskey(ENV, "GITPOD_REPO_ROOT"))
    run(```psql -f sqlsnippets/droptables.sql```)
else
    println("execute this in Your REPL")
    println("""run(```sudo -u postgres psql -f sqlsnippets/droptables.sql```)""")
end


psql:sqlsnippets/droptables.sql:1: ERROR:  table "schema_migrations" does not exist
psql:sqlsnippets/droptables.sql:2: ERROR:  table "tariffitemrevisions" does not exist
psql:sqlsnippets/droptables.sql:3: ERROR:  table "tariffitempartnerrefrevisions" does not exist
psql:sqlsnippets/droptables.sql:4: ERROR:  table "tariffitempartnerrefs" does not exist
psql:sqlsnippets/droptables.sql:5: ERROR:  table "tariffitems" does not exist
psql:sqlsnippets/droptables.sql:6: ERROR:  table "productitemrevisions" does not exist
psql:sqlsnippets/droptables.sql:7: ERROR:  table "tariffitemroles" does not exist
psql:sqlsnippets/droptables.sql:8: ERROR:  table "tariffitempartnerroles" does not exist
psql:sqlsnippets/droptables.sql:9: ERROR:  table "productitems" does not exist
psql:sqlsnippets/droptables.sql:10: ERROR:  table "contractpartnerrefrevisions" does not exist
psql:sqlsnippets/droptables.sql:11: ERROR:  table "contractpartnerrefs" does not exist
psql:sqlsnippets/droptables.sql:12: ERROR:  table

Process(`psql -f sqlsnippets/droptables.sql`, ProcessExited(0))

Loading the data model- Create tables, constraints etc. and load Roles

loading inverses of the role tables to provide role descriptions in object creation,  for instance like in: "ref_role=cpRole["Policy Holder"]

In [5]:
LifeInsuranceDataModel.load_model()

┌ Info: CREATE TABLE schema_migrations (version varchar(30))
└ @ SearchLightPostgreSQL /home/gitpod/.julia/packages/SearchLightPostgreSQL/cIJM3/src/SearchLightPostgreSQL.jl:138
┌ Info: Created table schema_migrations
└ @ SearchLightPostgreSQL /home/gitpod/.julia/packages/SearchLightPostgreSQL/cIJM3/src/SearchLightPostgreSQL.jl:309
┌ Info: CREATE TABLE histories (id BIGSERIAL PRIMARY KEY, dummy INTEGER )
└ @ SearchLightPostgreSQL /home/gitpod/.julia/packages/SearchLightPostgreSQL/cIJM3/src/SearchLightPostgreSQL.jl:138
┌ Info: CREATE TABLE versions (id BIGSERIAL PRIMARY KEY, ref_history BIGINT REFERENCES histories(id) ON DELETE CASCADE)
└ @ SearchLightPostgreSQL /home/gitpod/.julia/packages/SearchLightPostgreSQL/cIJM3/src/SearchLightPostgreSQL.jl:138
┌ Info: CREATE TABLE workflows (id BIGSERIAL PRIMARY KEY, ref_history BIGINT REFERENCES histories(id) ON DELETE CASCADE, ref_version BIGINT REFERENCES versions(id) ON DELETE CASCADE, tsw_validfrom TIMESTAMPTZ (3), tsdb_validfrom TIMESTAMPTZ 

1×4 Matrix{ProductPartRole}:
 ProductPartRole
| KEY            | VALUE                |
|----------------|----------------------|
| domain::String | ProductPart          |
| id::DbId       | 1                    |
| value::String  | Main Coverage - Life |
  …  ProductPartRole
| KEY            | VALUE                |
|----------------|----------------------|
| domain::String | ProductPart          |
| id::DbId       | 4                    |
| value::String  | Profit participation |


In [6]:
cpRole = Dict{String,Int64}()
map(find(LifeInsuranceDataModel.ContractPartnerRole)) do entry
    cpRole[entry.value] = entry.id.value
end
tiprRole = Dict{String,Int64}()
map(find(LifeInsuranceDataModel.TariffItemPartnerRole)) do entry
    tiprRole[entry.value] = entry.id.value
end
titrRole = Dict{String,Int64}()
map(find(LifeInsuranceDataModel.TariffItemRole)) do entry
    titrRole[entry.value] = entry.id.value
end

ppRole = Dict{String,Int64}()
map(find(LifeInsuranceDataModel.ProductPartRole)) do entry
    ppRole[entry.value] = entry.id.value
end


┌ Info: SELECT "contractpartnerroles"."id" AS "contractpartnerroles_id", "contractpartnerroles"."domain" AS "contractpartnerroles_domain", "contractpartnerroles"."value" AS "contractpartnerroles_value" FROM "contractpartnerroles" ORDER BY contractpartnerroles.id ASC
└ @ SearchLightPostgreSQL /home/gitpod/.julia/packages/SearchLightPostgreSQL/cIJM3/src/SearchLightPostgreSQL.jl:138
┌ Info: SELECT "tariffitempartnerroles"."id" AS "tariffitempartnerroles_id", "tariffitempartnerroles"."domain" AS "tariffitempartnerroles_domain", "tariffitempartnerroles"."value" AS "tariffitempartnerroles_value" FROM "tariffitempartnerroles" ORDER BY tariffitempartnerroles.id ASC
└ @ SearchLightPostgreSQL /home/gitpod/.julia/packages/SearchLightPostgreSQL/cIJM3/src/SearchLightPostgreSQL.jl:138
┌ Info: SELECT "tariffitemroles"."id" AS "tariffitemroles_id", "tariffitemroles"."domain" AS "tariffitemroles_domain", "tariffitemroles"."value" AS "tariffitemroles_value" FROM "tariffitemroles" ORDER BY tariffitemrole

4-element Vector{Int64}:
 1
 2
 3
 4

Or just connect to an existing model

In [7]:
LifeInsuranceDataModel.connect()

PostgreSQL connection (CONNECTION_OK) with parameters:
  user = gitpod
  passfile = /home/gitpod/.pgpass
  channel_binding = prefer
  dbname = postgres
  host = 127.0.0.1
  hostaddr = 127.0.0.1
  port = 5432
  client_encoding = UTF8
  options = -c DateStyle=ISO,YMD -c IntervalStyle=iso_8601 -c TimeZone=UTC
  application_name = LibPQ.jl
  sslmode = prefer
  sslcompression = 0
  sslsni = 1
  ssl_min_protocol_version = TLSv1.2
  gssencmode = prefer
  krbsrvname = postgres
  target_session_attrs = any

Create a Partner

In [8]:
p = LifeInsuranceDataModel.Partner()
pr = LifeInsuranceDataModel.PartnerRevision(description="Partner 1")
w = Workflow(
    tsw_validfrom=ZonedDateTime(2014, 5, 30, 21, 0, 1, 1, tz"Africa/Porto-Novo"),
)
create_entity!(w)
create_component!(p, pr, w)
commit_workflow!(w)

Partner1 = p.id.value

┌ Info: BEGIN
└ @ SearchLightPostgreSQL /home/gitpod/.julia/packages/SearchLightPostgreSQL/cIJM3/src/SearchLightPostgreSQL.jl:138
┌ Info: INSERT INTO histories ( "dummy" ) VALUES ( 0 ) RETURNING id
└ @ SearchLightPostgreSQL /home/gitpod/.julia/packages/SearchLightPostgreSQL/cIJM3/src/SearchLightPostgreSQL.jl:138
┌ Info: SELECT "histories"."id" AS "histories_id", "histories"."dummy" AS "histories_dummy" FROM "histories" WHERE "id" = 1 ORDER BY histories.id ASC
└ @ SearchLightPostgreSQL /home/gitpod/.julia/packages/SearchLightPostgreSQL/cIJM3/src/SearchLightPostgreSQL.jl:138
┌ Info: INSERT INTO versions ( "ref_history" ) VALUES ( E'1' ) RETURNING id
└ @ SearchLightPostgreSQL /home/gitpod/.julia/packages/SearchLightPostgreSQL/cIJM3/src/SearchLightPostgreSQL.jl:138
┌ Info: SELECT "versions"."id" AS "versions_id", "versions"."ref_history" AS "versions_ref_history" FROM "versions" WHERE "id" = 1 ORDER BY versions.id ASC
└ @ SearchLightPostgreSQL /home/gitpod/.julia/packages/SearchLightPostgr

1

Create tariffs

In [9]:
# create Tariffs
function create_tariff(dsc,mt)
    t = LifeInsuranceDataModel.Tariff()
    tr = LifeInsuranceDataModel.TariffRevision(description=dsc,mortality_table=mt)
    w = Workflow(
        tsw_validfrom=ZonedDateTime(2014, 5, 30, 21, 0, 1, 1, tz"Africa/Porto-Novo"),
    )
    create_entity!(w)
    create_component!(t, tr, w)
    commit_workflow!(w)
    t.id.value
end

LifeRiskTariff = create_tariff("Life Risk Insurance", "1980 CET - Male Nonsmoker, ANB")
TerminalIllnessTariff = create_tariff("Terminal Illness", "2001 VBT Residual Standard Select and Ultimate - Male Nonsmoker, ANB")
OccupationalDisabilityTariff = create_tariff("Occupational Disability", "2001 VBT Residual Standard Select and Ultimate - Male Nonsmoker, ANB")
ProfitParticipationTariff = create_tariff("Profit participation", "2001 VBT Residual Standard Select and Ultimate - Male Nonsmoker, ANB")
LifeRiskTariff2 = create_tariff("Life Risk Insurance", "2001 VBT Residual Standard Select and Ultimate - Male Nonsmoker, ANB")

find(TariffRevision)

┌ Info: BEGIN
└ @ SearchLightPostgreSQL /home/gitpod/.julia/packages/SearchLightPostgreSQL/cIJM3/src/SearchLightPostgreSQL.jl:138
┌ Info: INSERT INTO histories ( "dummy" ) VALUES ( 0 ) RETURNING id
└ @ SearchLightPostgreSQL /home/gitpod/.julia/packages/SearchLightPostgreSQL/cIJM3/src/SearchLightPostgreSQL.jl:138
┌ Info: SELECT "histories"."id" AS "histories_id", "histories"."dummy" AS "histories_dummy" FROM "histories" WHERE "id" = 2 ORDER BY histories.id ASC
└ @ SearchLightPostgreSQL /home/gitpod/.julia/packages/SearchLightPostgreSQL/cIJM3/src/SearchLightPostgreSQL.jl:138
┌ Info: INSERT INTO versions ( "ref_history" ) VALUES ( E'2' ) RETURNING id
└ @ SearchLightPostgreSQL /home/gitpod/.julia/packages/SearchLightPostgreSQL/cIJM3/src/SearchLightPostgreSQL.jl:138
┌ Info: SELECT "versions"."id" AS "versions_id", "versions"."ref_history" AS "versions_ref_history" FROM "versions" WHERE "id" = 2 ORDER BY versions.id ASC
└ @ SearchLightPostgreSQL /home/gitpod/.julia/packages/SearchLightPostgr

5-element Vector{TariffRevision}:
 TariffRevision
| KEY                     | VALUE                          |
|-------------------------|--------------------------------|
| description::String     | Life Risk Insurance            |
| id::DbId                | 1                              |
| mortality_table::String | 1980 CET - Male Nonsmoker, ANB |
| ref_component::DbId     | 1                              |
| ref_invalidfrom::DbId   | 9223372036854775807            |
| ref_validfrom::DbId     | 2                              |

 TariffRevision
| KEY                     | VALUE                                                                |
|-------------------------|----------------------------------------------------------------------|
| description::String     | Terminal Illness                                                     |
| id::DbId                | 2                                                                    |
| mortality_table::String | 2001 VBT Residual Sta

In [10]:
find(Tariff,SQLWhereExpression("id=?",ProfitParticipationTariff))
find(Tariff, SQLWhereExpression("id=?", TerminalIllnessTariff))

┌ Info: SELECT "tariffs"."id" AS "tariffs_id", "tariffs"."ref_history" AS "tariffs_ref_history", "tariffs"."ref_version" AS "tariffs_ref_version" FROM "tariffs" WHERE id=4 ORDER BY tariffs.id ASC
└ @ SearchLightPostgreSQL /home/gitpod/.julia/packages/SearchLightPostgreSQL/cIJM3/src/SearchLightPostgreSQL.jl:138
┌ Info: SELECT "tariffs"."id" AS "tariffs_id", "tariffs"."ref_history" AS "tariffs_ref_history", "tariffs"."ref_version" AS "tariffs_ref_version" FROM "tariffs" WHERE id=2 ORDER BY tariffs.id ASC
└ @ SearchLightPostgreSQL /home/gitpod/.julia/packages/SearchLightPostgreSQL/cIJM3/src/SearchLightPostgreSQL.jl:138


1-element Vector{Tariff}:
 Tariff
| KEY               | VALUE |
|-------------------|-------|
| id::DbId          | 2     |
| ref_history::DbId | 3     |
| ref_version::DbId | 3     |


Create Product

In [11]:
p = Product()
pr = ProductRevision(description="Life Risk")

pp = ProductPart()
ppr = ProductPartRevision(ref_tariff=LifeRiskTariff, ref_role=ppRole["Main Coverage - Life"], description="Main Coverage - Life")

pp2 = ProductPart()
ppr2 = ProductPartRevision(ref_tariff=ProfitParticipationTariff, ref_role=ppRole["Profit participation"], description="Profit participation Lif Risk")

w0 = Workflow(
    tsw_validfrom=ZonedDateTime(2014, 5, 30, 21, 0, 1, 1, tz"Africa/Porto-Novo"),
)
create_entity!(w0)
create_component!(p, pr, w0)
create_subcomponent!(p, pp, ppr, w0)
create_subcomponent!(p, pp2, ppr2, w0)
commit_workflow!(w0)

LifeRiskProduct = p.id.value
println(LifeRiskProduct)

p = Product()
pr = ProductRevision(description="Life Risk - Terminal Illness")

pp = ProductPart()
ppr = ProductPartRevision(ref_tariff=LifeRiskTariff, ref_role=ppRole["Main Coverage - Life"], description="Main Coverage - Life")

pp2 = ProductPart()
ppr2 = ProductPartRevision(ref_tariff=ProfitParticipationTariff, ref_role=ppRole["Profit participation"], description="Profit participation Life Risk")

pp3 = ProductPart()
ppr3 = ProductPartRevision(ref_tariff=TerminalIllnessTariff, ref_role=ppRole["Supplementary Coverage - Terminal Illness"], description="additional cover Terminal Illness")

pp4 = ProductPart()
ppr4 = ProductPartRevision(ref_tariff=ProfitParticipationTariff, ref_role=ppRole["Profit participation"], description="Profit participation Terminal Illness")

pp5 = ProductPart()
ppr5 = ProductPartRevision(ref_tariff=OccupationalDisabilityTariff, ref_role=ppRole["Supplementary Coverage - Occupational Disablity"], description="additional cover Occupational Disablity")

pp6 = ProductPart()
ppr6 = ProductPartRevision(ref_tariff=ProfitParticipationTariff, ref_role=ppRole["Profit participation"], description="Profit participation Occ.Disablity")



w0 = Workflow(
    tsw_validfrom=ZonedDateTime(2014, 5, 30, 21, 0, 1, 1, tz"Africa/Porto-Novo"),
)
create_entity!(w0)
create_component!(p, pr, w0)
create_subcomponent!(p, pp, ppr, w0)
create_subcomponent!(p, pp2, ppr2, w0)
create_subcomponent!(p, pp3, ppr3, w0)
create_subcomponent!(p, pp4, ppr4, w0)
create_subcomponent!(p, pp5, ppr5, w0)
create_subcomponent!(p, pp6, ppr6, w0)
commit_workflow!(w0)

LifeRiskTIODProduct = p.id.value
println(LifeRiskTIODProduct)

┌ Info: BEGIN
└ @ SearchLightPostgreSQL /home/gitpod/.julia/packages/SearchLightPostgreSQL/cIJM3/src/SearchLightPostgreSQL.jl:138
┌ Info: INSERT INTO histories ( "dummy" ) VALUES ( 0 ) RETURNING id
└ @ SearchLightPostgreSQL /home/gitpod/.julia/packages/SearchLightPostgreSQL/cIJM3/src/SearchLightPostgreSQL.jl:138
┌ Info: SELECT "histories"."id" AS "histories_id", "histories"."dummy" AS "histories_dummy" FROM "histories" WHERE "id" = 7 ORDER BY histories.id ASC
└ @ SearchLightPostgreSQL /home/gitpod/.julia/packages/SearchLightPostgreSQL/cIJM3/src/SearchLightPostgreSQL.jl:138
┌ Info: INSERT INTO versions ( "ref_history" ) VALUES ( E'7' ) RETURNING id
└ @ SearchLightPostgreSQL /home/gitpod/.julia/packages/SearchLightPostgreSQL/cIJM3/src/SearchLightPostgreSQL.jl:138
┌ Info: SELECT "versions"."id" AS "versions_id", "versions"."ref_history" AS "versions_ref_history" FROM "versions" WHERE "id" = 7 ORDER BY versions.id ASC
└ @ SearchLightPostgreSQL /home/gitpod/.julia/packages/SearchLightPostgr

1


┌ Info: COMMIT
└ @ SearchLightPostgreSQL /home/gitpod/.julia/packages/SearchLightPostgreSQL/cIJM3/src/SearchLightPostgreSQL.jl:138
┌ Info: BEGIN
└ @ SearchLightPostgreSQL /home/gitpod/.julia/packages/SearchLightPostgreSQL/cIJM3/src/SearchLightPostgreSQL.jl:138
┌ Info: INSERT INTO products ( "ref_history", "ref_version" ) VALUES ( E'8', E'8' ) RETURNING id
└ @ SearchLightPostgreSQL /home/gitpod/.julia/packages/SearchLightPostgreSQL/cIJM3/src/SearchLightPostgreSQL.jl:138
┌ Info: SELECT "products"."id" AS "products_id", "products"."ref_history" AS "products_ref_history", "products"."ref_version" AS "products_ref_version" FROM "products" WHERE "id" = 2 ORDER BY products.id ASC
└ @ SearchLightPostgreSQL /home/gitpod/.julia/packages/SearchLightPostgreSQL/cIJM3/src/SearchLightPostgreSQL.jl:138
┌ Info: INSERT INTO productrevisions ( "ref_component", "ref_validfrom", "ref_invalidfrom", "description" ) VALUES ( E'2', E'8', E'9223372036854775807', E'Life Risk - Terminal Illness' ) RETURNING id
└ 

2


┌ Info: UPDATE workflows SET  "id" = E'8', "ref_history" = E'8', "ref_version" = E'8', "tsdb_validfrom" = E'2022-08-25T15:39:38.646+01:00', "tsw_validfrom" = E'2014-05-30T20:00:01.001+00:00', "is_committed" = 1 WHERE workflows.id = '8' RETURNING id
└ @ SearchLightPostgreSQL /home/gitpod/.julia/packages/SearchLightPostgreSQL/cIJM3/src/SearchLightPostgreSQL.jl:138
┌ Info: SELECT "workflows"."id" AS "workflows_id", "workflows"."ref_history" AS "workflows_ref_history", "workflows"."ref_version" AS "workflows_ref_version", "workflows"."tsdb_validfrom" AS "workflows_tsdb_validfrom", "workflows"."tsw_validfrom" AS "workflows_tsw_validfrom", "workflows"."is_committed" AS "workflows_is_committed" FROM "workflows" WHERE "id" = 8 ORDER BY workflows.id ASC
└ @ SearchLightPostgreSQL /home/gitpod/.julia/packages/SearchLightPostgreSQL/cIJM3/src/SearchLightPostgreSQL.jl:138
┌ Info: COMMIT
└ @ SearchLightPostgreSQL /home/gitpod/.julia/packages/SearchLightPostgreSQL/cIJM3/src/SearchLightPostgreSQL.jl:13

Testing

Create contract

In [12]:
    w1 = Workflow(
        tsw_validfrom=ZonedDateTime(2014, 5, 30, 21, 0, 1, 1, tz"Africa/Porto-Novo"),
    )
    
    create_entity!(w1)
    c = Contract()
    cr = ContractRevision(description="contract creation properties")
    create_component!(c, cr, w1)
    
    cpr = ContractPartnerRef(ref_super=c.id)
    cprr = ContractPartnerRefRevision(ref_partner=Partner1, ref_role=cpRole["Policy Holder"], description="policiyholder ref properties")
    create_subcomponent!(c, cpr, cprr, w1)
    # pi 1
    cpi = ProductItem(ref_super=c.id)
    cpir = ProductItemRevision(position=1, ref_product=LifeRiskTIODProduct, description="from contract creation")
    create_subcomponent!(c, cpi, cpir, w1)
    # pi 1 ti 1 
    pit = TariffItem(ref_super=cpi.id)
    pitr = TariffItemRevision(ref_tariff=LifeRiskTariff, ref_role=titrRole["Main Coverage - Life"], description="Life Risk tariff parameters")
    create_subcomponent!(cpi, pit, pitr, w1)
    # pi 1 ti 1 p 1
    pitp = TariffItemPartnerRef(ref_super=pit.id)
    pitpr = TariffItemPartnerRefRevision(ref_partner=Partner1, ref_role=tiprRole["Insured Person"], description="partner 1 ref properties")
    create_subcomponent!(pit, pitp, pitpr, w1)
    
    # pi 1 ti 2 
    pit = TariffItem(ref_super=cpi.id)
    pitr = TariffItemRevision(ref_tariff=ProfitParticipationTariff, ref_role=titrRole["Profit participation"], description="Profit participation tariff parameters")
    create_subcomponent!(cpi, pit, pitr, w1)
    # pi 1 ti 2 p 1
    pitp = TariffItemPartnerRef(ref_super=pit.id)
    pitpr = TariffItemPartnerRefRevision(ref_partner=Partner1, ref_role=tiprRole["Insured Person"], description="partner 1 ref properties")
    create_subcomponent!(pit, pitp, pitpr, w1)
    
    # pi 1 ti 3 
    pit = TariffItem(ref_super=cpi.id)
    pitr = TariffItemRevision(ref_tariff=TerminalIllnessTariff, ref_role=titrRole["Supplementary Coverage - Terminal Illness"], description="Terminal Illness tariff parameters")
    create_subcomponent!(cpi, pit, pitr, w1)
    # pi 1 ti 3 p 1
    pitp = TariffItemPartnerRef(ref_super=pit.id)
    pitpr = TariffItemPartnerRefRevision(ref_partner=Partner1, ref_role=tiprRole["Insured Person"], description="partner 1 ref properties")
    create_subcomponent!(pit, pitp, pitpr, w1)
    
    # pi 1 ti 4 
    pit = TariffItem(ref_super=cpi.id)
    pitr = TariffItemRevision(ref_tariff=ProfitParticipationTariff, ref_role=titrRole["Profit participation"], description="Profitparticipation tariff parameters")
    create_subcomponent!(cpi, pit, pitr, w1)
    # pi 1 ti 4 p 1
    pitp = TariffItemPartnerRef(ref_super=pit.id)
    pitpr = TariffItemPartnerRefRevision(ref_partner=Partner1, ref_role=tiprRole["Insured Person"], description="partner 1 ref properties")
    create_subcomponent!(pit, pitp, pitpr, w1)
    
    
    # pi 1 ti 5 
    pit = TariffItem(ref_super=cpi.id)
    pitr = TariffItemRevision(ref_tariff=ProfitParticipationTariff, ref_role=titrRole["Supplementary Coverage - Occupational Disablity"], description="Occupational Disablity tariff parameters")
    create_subcomponent!(cpi, pit, pitr, w1)
    # pi 1 ti 5 p 1
    pitp = TariffItemPartnerRef(ref_super=pit.id)
    pitpr = TariffItemPartnerRefRevision(ref_partner=Partner1, ref_role=tiprRole["Insured Person"], description="partner 1 ref properties")
    create_subcomponent!(pit, pitp, pitpr, w1)
    
    # pi 1 ti 6 
    pit = TariffItem(ref_super=cpi.id)
    pitr = TariffItemRevision(ref_tariff=ProfitParticipationTariff, ref_role=titrRole["Profit participation"], description="Profit Participation tariff parameters")
    create_subcomponent!(cpi, pit, pitr, w1)
    # pi 1 ti 6 p 1
    pitp = TariffItemPartnerRef(ref_super=pit.id)
    pitpr = TariffItemPartnerRefRevision(ref_partner=Partner1, ref_role=tiprRole["Insured Person"], description="partner 1 ref properties")
    create_subcomponent!(pit, pitp, pitpr, w1)
    
    
    
    
    commit_workflow!(w1)

┌ Info: BEGIN
└ @ SearchLightPostgreSQL /home/gitpod/.julia/packages/SearchLightPostgreSQL/cIJM3/src/SearchLightPostgreSQL.jl:138
┌ Info: INSERT INTO histories ( "dummy" ) VALUES ( 0 ) RETURNING id
└ @ SearchLightPostgreSQL /home/gitpod/.julia/packages/SearchLightPostgreSQL/cIJM3/src/SearchLightPostgreSQL.jl:138
┌ Info: SELECT "histories"."id" AS "histories_id", "histories"."dummy" AS "histories_dummy" FROM "histories" WHERE "id" = 9 ORDER BY histories.id ASC
└ @ SearchLightPostgreSQL /home/gitpod/.julia/packages/SearchLightPostgreSQL/cIJM3/src/SearchLightPostgreSQL.jl:138
┌ Info: INSERT INTO versions ( "ref_history" ) VALUES ( E'9' ) RETURNING id
└ @ SearchLightPostgreSQL /home/gitpod/.julia/packages/SearchLightPostgreSQL/cIJM3/src/SearchLightPostgreSQL.jl:138
┌ Info: SELECT "versions"."id" AS "versions_id", "versions"."ref_history" AS "versions_ref_history" FROM "versions" WHERE "id" = 9 ORDER BY versions.id ASC
└ @ SearchLightPostgreSQL /home/gitpod/.julia/packages/SearchLightPostgr

update Contract yellow

In [13]:
cr1 = ContractRevision(ref_component=c.id, description="contract 1, 2nd mutation")
w2 = Workflow(
    ref_history=w1.ref_history,
    tsw_validfrom=ZonedDateTime(2016, 5, 30, 21, 0, 1, 1, tz"Africa/Porto-Novo"),
)
update_entity!(w2)
update_component!(cr, cr1, w2)
commit_workflow!(w2)

┌ Info: BEGIN
└ @ SearchLightPostgreSQL /home/gitpod/.julia/packages/SearchLightPostgreSQL/cIJM3/src/SearchLightPostgreSQL.jl:138
┌ Info: INSERT INTO versions ( "ref_history" ) VALUES ( E'9' ) RETURNING id
└ @ SearchLightPostgreSQL /home/gitpod/.julia/packages/SearchLightPostgreSQL/cIJM3/src/SearchLightPostgreSQL.jl:138
┌ Info: SELECT "versions"."id" AS "versions_id", "versions"."ref_history" AS "versions_ref_history" FROM "versions" WHERE "id" = 10 ORDER BY versions.id ASC
└ @ SearchLightPostgreSQL /home/gitpod/.julia/packages/SearchLightPostgreSQL/cIJM3/src/SearchLightPostgreSQL.jl:138
┌ Info: INSERT INTO workflows ( "ref_history", "ref_version", "tsdb_validfrom", "tsw_validfrom", "is_committed" ) VALUES ( E'9', E'10', E'2038-01-19T03:14:06.999+00:00', E'2016-05-30T21:00:01.001+01:00', 0 ) RETURNING id
└ @ SearchLightPostgreSQL /home/gitpod/.julia/packages/SearchLightPostgreSQL/cIJM3/src/SearchLightPostgreSQL.jl:138
┌ Info: SELECT "workflows"."id" AS "workflows_id", "workflows"."ref_

   update Contract red

In [14]:

cr2 = ContractRevision(ref_component=c.id, description="contract 1, 3rd mutation retrospective")
w3 = Workflow(
    ref_history=w2.ref_history,
    tsw_validfrom=ZonedDateTime(2015, 5, 30, 21, 0, 1, 1, tz"Africa/Porto-Novo"),
)
update_entity!(w3)
update_component!(cr1, cr2, w3)


commit_workflow!(w3)


┌ Info: BEGIN
└ @ SearchLightPostgreSQL /home/gitpod/.julia/packages/SearchLightPostgreSQL/cIJM3/src/SearchLightPostgreSQL.jl:138
┌ Info: INSERT INTO versions ( "ref_history" ) VALUES ( E'9' ) RETURNING id
└ @ SearchLightPostgreSQL /home/gitpod/.julia/packages/SearchLightPostgreSQL/cIJM3/src/SearchLightPostgreSQL.jl:138
┌ Info: SELECT "versions"."id" AS "versions_id", "versions"."ref_history" AS "versions_ref_history" FROM "versions" WHERE "id" = 11 ORDER BY versions.id ASC
└ @ SearchLightPostgreSQL /home/gitpod/.julia/packages/SearchLightPostgreSQL/cIJM3/src/SearchLightPostgreSQL.jl:138
┌ Info: INSERT INTO workflows ( "ref_history", "ref_version", "tsdb_validfrom", "tsw_validfrom", "is_committed" ) VALUES ( E'9', E'11', E'2038-01-19T03:14:06.999+00:00', E'2015-05-30T21:00:01.001+01:00', 0 ) RETURNING id
└ @ SearchLightPostgreSQL /home/gitpod/.julia/packages/SearchLightPostgreSQL/cIJM3/src/SearchLightPostgreSQL.jl:138
┌ Info: SELECT "workflows"."id" AS "workflows_id", "workflows"."ref_

In [15]:
w4 = Workflow(
    ref_history=w2.ref_history,
    tsw_validfrom=ZonedDateTime(2018, 5, 30, 21, 0, 1, 1, tz"Africa/Porto-Novo"),
)
update_entity!(w4)
cr3 = ContractRevision(ref_component=c.id, description="contract 1, 4th mutation")
update_component!(cr2, cr3, w4)

# pi 2
LifeRiskTIODProduct = find(Product, SQLWhereExpression("id=?", 2))[1].id.value
Partner1
PartnerRole = tiprRole["Insured Person"]

cpi = ProductItem(ref_super=c.id)
cpir = ProductItemRevision(position=2, ref_product=LifeRiskTIODProduct, description="from contract 4th mutation")
create_subcomponent!(c, cpi, cpir, w4)

println("vor neuem Teil")



map(find(ProductPart, SQLWhereExpression("ref_super=?", LifeRiskTIODProduct))) do pp
    println(pp.id.value)
    map(find(ProductPartRevision, SQLWhereExpression("ref_component=?", pp.id.value))) do ppr
        println(ppr.description)
        ti=TariffItem(ref_super=cpi.id)
        tir = TariffItemRevision(ref_role=ppr.ref_role, ref_tariff=ppr.ref_tariff, description=ppr.description)
        create_subcomponent!(cpi, ti, tir, w4)
        tip = TariffItemPartnerRef(ref_super=cpi.id)
        tipr = TariffItemPartnerRefRevision(ref_partner=Partner1, ref_role=PartnerRole)
        create_subcomponent!(cpi, tip, tipr, w4)
        println(tir)
        println(tipr)
    end
end
println("vor rollback")
# rollback_workflow!(w4)
commit_workflow!(w4)

┌ Info: BEGIN
└ @ SearchLightPostgreSQL /home/gitpod/.julia/packages/SearchLightPostgreSQL/cIJM3/src/SearchLightPostgreSQL.jl:138
┌ Info: INSERT INTO versions ( "ref_history" ) VALUES ( E'9' ) RETURNING id
└ @ SearchLightPostgreSQL /home/gitpod/.julia/packages/SearchLightPostgreSQL/cIJM3/src/SearchLightPostgreSQL.jl:138
┌ Info: SELECT "versions"."id" AS "versions_id", "versions"."ref_history" AS "versions_ref_history" FROM "versions" WHERE "id" = 12 ORDER BY versions.id ASC
└ @ SearchLightPostgreSQL /home/gitpod/.julia/packages/SearchLightPostgreSQL/cIJM3/src/SearchLightPostgreSQL.jl:138
┌ Info: INSERT INTO workflows ( "ref_history", "ref_version", "tsdb_validfrom", "tsw_validfrom", "is_committed" ) VALUES ( E'9', E'12', E'2038-01-19T03:14:06.999+00:00', E'2018-05-30T21:00:01.001+01:00', 0 ) RETURNING id
└ @ SearchLightPostgreSQL /home/gitpod/.julia/packages/SearchLightPostgreSQL/cIJM3/src/SearchLightPostgreSQL.jl:138
┌ Info: SELECT "workflows"."id" AS "workflows_id", "workflows"."ref_

vor neuem Teil


┌ Info: SELECT "productparts"."id" AS "productparts_id", "productparts"."ref_history" AS "productparts_ref_history", "productparts"."ref_version" AS "productparts_ref_version", "productparts"."ref_super" AS "productparts_ref_super" FROM "productparts" WHERE ref_super=2 ORDER BY productparts.id ASC
└ @ SearchLightPostgreSQL /home/gitpod/.julia/packages/SearchLightPostgreSQL/cIJM3/src/SearchLightPostgreSQL.jl:138


3


┌ Info: SELECT "productpartrevisions"."id" AS "productpartrevisions_id", "productpartrevisions"."ref_component" AS "productpartrevisions_ref_component", "productpartrevisions"."ref_validfrom" AS "productpartrevisions_ref_validfrom", "productpartrevisions"."ref_invalidfrom" AS "productpartrevisions_ref_invalidfrom", "productpartrevisions"."ref_tariff" AS "productpartrevisions_ref_tariff", "productpartrevisions"."ref_role" AS "productpartrevisions_ref_role", "productpartrevisions"."description" AS "productpartrevisions_description" FROM "productpartrevisions" WHERE ref_component=3 ORDER BY productpartrevisions.id ASC
└ @ SearchLightPostgreSQL /home/gitpod/.julia/packages/SearchLightPostgreSQL/cIJM3/src/SearchLightPostgreSQL.jl:138


...

┌ Info: SELECT "productpartrevisions"."id" AS "productpartrevisions_id", "productpartrevisions"."ref_component" AS "productpartrevisions_ref_component", "productpartrevisions"."ref_validfrom" AS "productpartrevisions_ref_validfrom", "productpartrevisions"."ref_invalidfrom" AS "productpartrevisions_ref_invalidfrom", "productpartrevisions"."ref_tariff" AS "productpartrevisions_ref_tariff", "productpartrevisions"."ref_role" AS "productpartrevisions_ref_role", "productpartrevisions"."description" AS "productpartrevisions_description" FROM "productpartrevisions" WHERE ref_component=4 ORDER BY productpartrevisions.id ASC
└ @ SearchLightPostgreSQL /home/gitpod/.julia/packages/SearchLightPostgreSQL/cIJM3/src/SearchLightPostgreSQL.jl:138
┌ Info: BEGIN
└ @ SearchLightPostgreSQL /home/gitpod/.julia/packages/SearchLightPostgreSQL/cIJM3/src/SearchLightPostgreSQL.jl:138
┌ Info: INSERT INTO tariffitems ( "ref_history", "ref_version", "ref_super" ) VALUES ( E'9', E'9223372036854775807', E'2' ) RETURNI

TariffItemPartnerRefRevision(ref_validfrom::DbId=12,ref_invalidfrom::DbId=9223372036854775807,id::DbId=7,ref_partner::DbId=1,ref_component::DbId=7,ref_role::DbId=1,description::String=)
4
Profit participation Life Risk
TariffItemRevision(ref_component::DbId=8,ref_role::DbId=4,ref_tariff::DbId=4,annuity_due::Float64=0.0,ref_validfrom::DbId=12,net_premium::Float64=0.0,annuity_immediate::Float64=0.0,description::String=Profit participation Life Risk,id::DbId=8,deferment::Int64=0,ref_invalidfrom::DbId=9223372036854775807)
TariffItemPartnerRefRevision(ref_validfrom::DbId=12,ref_invalidfrom::DbId=9223372036854775807,id::DbId=8,ref_partner::DbId=1,ref_component::DbId=8,ref_role::DbId=1,description::String=)
5
additional cover Terminal Illness
TariffItemRevision(ref_component::DbId=9,ref_role::DbId=3,ref_tariff::DbId=2,annuity_due::Float64=0.0,ref_validfrom::DbId=12,net_premium::Float64=0.0,annuity_immediate::Float64=0.0,description::String=additional cover Terminal Illness,id::DbId=9,defermen

┌ Info: COMMIT
└ @ SearchLightPostgreSQL /home/gitpod/.julia/packages/SearchLightPostgreSQL/cIJM3/src/SearchLightPostgreSQL.jl:138
┌ Info: SELECT "productpartrevisions"."id" AS "productpartrevisions_id", "productpartrevisions"."ref_component" AS "productpartrevisions_ref_component", "productpartrevisions"."ref_validfrom" AS "productpartrevisions_ref_validfrom", "productpartrevisions"."ref_invalidfrom" AS "productpartrevisions_ref_invalidfrom", "productpartrevisions"."ref_tariff" AS "productpartrevisions_ref_tariff", "productpartrevisions"."ref_role" AS "productpartrevisions_ref_role", "productpartrevisions"."description" AS "productpartrevisions_description" FROM "productpartrevisions" WHERE ref_component=8 ORDER BY productpartrevisions.id ASC
└ @ SearchLightPostgreSQL /home/gitpod/.julia/packages/SearchLightPostgreSQL/cIJM3/src/SearchLightPostgreSQL.jl:138
┌ Info: BEGIN
└ @ SearchLightPostgreSQL /home/gitpod/.julia/packages/SearchLightPostgreSQL/cIJM3/src/SearchLightPostgreSQL.jl:138


TariffItemRevision(ref_component::DbId=12,ref_role::DbId=4,ref_tariff::DbId=4,annuity_due::Float64=0.0,ref_validfrom::DbId=12,net_premium::Float64=0.0,annuity_immediate::Float64=0.0,description::String=Profit participation Occ.Disablity,id::DbId=12,deferment::Int64=0,ref_invalidfrom::DbId=9223372036854775807)
TariffItemPartnerRefRevision(ref_validfrom::DbId=12,ref_invalidfrom::DbId=9223372036854775807,id::DbId=12,ref_partner::DbId=1,ref_component::DbId=12,ref_role::DbId=1,description::String=)
vor rollback


┌ Info: COMMIT
└ @ SearchLightPostgreSQL /home/gitpod/.julia/packages/SearchLightPostgreSQL/cIJM3/src/SearchLightPostgreSQL.jl:138


In [16]:
using BitemporalPostgres
history = 9
txntime = MaxDate

res = SearchLight.query("select s.tsdb_invalidfrom as sdbinv, m.tsdb_validfrom as mdbval, 
      m.tsworld_validfrom as mwval, s.tsworld_validfrom as swval,m.tsworld_invalidfrom as mwinv,s.tsworld_invalidfrom as swinv, 
      m.id as mid, s.id as sid , m.ref_history as mh , m.ref_version as mv , s.ref_version as sv
      from validityintervals m join validityintervals s 
      on m.ref_history=s.ref_history
      and m.ref_version != s.ref_version
      and m.tsdb_validfrom = s.tsdb_invalidfrom
      and m.tsworld_validfrom <= s.tsworld_validfrom 
      --and tstzrange(m.tsworld_validfrom, m.tsworld_invalidfrom) @> s.tsworld_validfrom -- tstzrange(s.tsworld_validfrom,s.tsworld_invalidfrom)
      where m.ref_history=$history
      and m.tsdb_invalidfrom = TIMESTAMPTZ '$txntime'")

println(res)


┌ Info: select s.tsdb_invalidfrom as sdbinv, m.tsdb_validfrom as mdbval, 
│       m.tsworld_validfrom as mwval, s.tsworld_validfrom as swval,m.tsworld_invalidfrom as mwinv,s.tsworld_invalidfrom as swinv, 
│       m.id as mid, s.id as sid , m.ref_history as mh , m.ref_version as mv , s.ref_version as sv
│       from validityintervals m join validityintervals s 
│       on m.ref_history=s.ref_history
│       and m.ref_version != s.ref_version
│       and m.tsdb_validfrom = s.tsdb_invalidfrom
│       and m.tsworld_validfrom <= s.tsworld_validfrom 
│       --and tstzrange(m.tsworld_validfrom, m.tsworld_invalidfrom) @> s.tsworld_validfrom -- tstzrange(s.tsworld_validfrom,s.tsworld_invalidfrom)
│       where m.ref_history=9
│       and m.tsdb_invalidfrom = TIMESTAMPTZ '2038-01-19T03:14:06.999+00:00'
└ @ SearchLightPostgreSQL /home/gitpod/.julia/packages/SearchLightPostgreSQL/cIJM3/src/SearchLightPostgreSQL.jl:138


1×11 DataFrame
 Row │ sdbinv                         mdbval                         mwval                          swval                          mwinv                          swinv                          mid     sid     mh      mv      sv     
     │ ZonedDat…?                     ZonedDat…?                     ZonedDat…?                     ZonedDat…?                     ZonedDat…?                     ZonedDat…?                     Int64?  Int64?  Int64?  Int64?  Int64? 
─────┼──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
   1 │ 2022-08-25T14:39:55.343+00:00  2022-08-25T14:39:55.343+00:00  2014-05-30T20:00:01.001+00:00  2016-05-30T20:00:01.001+00:00  2015-05-30T20:00:01.001+00:00  2038-01-19T03:14:06.999+00:00      13      10       9       9      10


In [17]:
txns = SearchLight.query(
    "select tsdb_validfrom as vf from validityintervals union
    select tsdb_invalidfrom as vf from validityintervals 
    group by vf order by vf")

refs = SearchLight.query(
    "select tsworld_validfrom as vf from validityintervals union
    select tsworld_invalidfrom as vf from validityintervals 
    group by vf order by vf")

println(txns)

println(refs)

txnDict = Dict()
for i = 1:first(size(txns))
    txnDict[txns[i, 1]] = i
end


refDict = Dict()
for i = 1:first(size(refs))
    refDict[refs[i, 1]] = i
end

println(txnDict)

println(refDict)

using BitemporalPostgres
valints = find(ValidityInterval)


vi = valints[1]


13×1 DataFrame
 Row │ vf                            
     │ ZonedDat…?                    
─────┼───────────────────────────────
   1 │ 2022-08-25T14:39:25.912+00:00
   2 │ 2022-08-25T14:39:29.755+00:00
   3 │ 2022-08-25T14:39:29.794+00:00
   4 │ 2022-08-25T14:39:29.818+00:00
   5 │ 2022-08-25T14:39:29.845+00:00
   6 │ 2022-08-25T14:39:29.878+00:00
   7 │ 2022-08-25T14:39:38.537+00:00
   8 │ 2022-08-25T14:39:38.646+00:00
   9 │ 2022-08-25T14:39:54.747+00:00
  10 │ 2022-08-25T14:39:55.193+00:00
  11 │ 2022-08-25T14:39:55.343+00:00
  12 │ 2022-08-25T14:39:56.297+00:00
  13 │ 2038-01-19T03:14:06.999+00:00
5×1 DataFrame
 Row │ vf                            
     │ ZonedDat…?                    
─────┼───────────────────────────────
   1 │ 2014-05-30T20:00:01.001+00:00
   2 │ 2015-05-30T20:00:01.001+00:00
   3 │ 2016-05-30T20:00:01.001+00:00
   4 │ 2018-05-30T20:00:01.001+00:00
   5 │ 2038-01-19T03:14:06.999+00:00


┌ Info: select tsdb_validfrom as vf from validityintervals union
│     select tsdb_invalidfrom as vf from validityintervals 
│     group by vf order by vf
└ @ SearchLightPostgreSQL /home/gitpod/.julia/packages/SearchLightPostgreSQL/cIJM3/src/SearchLightPostgreSQL.jl:138
┌ Info: select tsworld_validfrom as vf from validityintervals union
│     select tsworld_invalidfrom as vf from validityintervals 
│     group by vf order by vf
└ @ SearchLightPostgreSQL /home/gitpod/.julia/packages/SearchLightPostgreSQL/cIJM3/src/SearchLightPostgreSQL.jl:138


Dict{Any, Any}(ZonedDateTime(2022, 8, 25, 14, 39, 38, 646, tz"UTC") => 8, ZonedDateTime(2022, 8, 25, 14, 39, 38, 537, tz"UTC") => 7, ZonedDateTime(2038, 1, 19, 3, 14, 6, 999, tz"UTC") => 13, ZonedDateTime(2022, 8, 25, 14, 39, 56, 297, tz"UTC") => 12, ZonedDateTime(2022, 8, 25, 14, 39, 29, 794, tz"UTC") => 3, ZonedDateTime(2022, 8, 25, 14, 39, 55, 193, tz"UTC") => 10, ZonedDateTime(2022, 8, 25, 14, 39, 29, 845, tz"UTC") => 5, ZonedDateTime(2022, 8, 25, 14, 39, 25, 912, tz"UTC") => 1, ZonedDateTime(2022, 8, 25, 14, 39, 54, 747, tz"UTC") => 9, ZonedDateTime(2022, 8, 25, 14, 39, 29, 818, tz"UTC") => 4, ZonedDateTime(2022, 8, 25, 14, 39, 29, 878, tz"UTC") => 6, ZonedDateTime(2022, 8, 25, 14, 39, 55, 343, tz"UTC") => 11, ZonedDateTime(2022, 8, 25, 14, 39, 29, 755, tz"UTC") => 2)
Dict{Any, Any}(ZonedDateTime(2016, 5, 30, 20, 0, 1, 1, tz"UTC") => 3, ZonedDateTime(2014, 5, 30, 20, 0, 1, 1, tz"UTC") => 1, ZonedDateTime(2038, 1, 19, 3, 14, 6, 999, tz"UTC") => 5, ZonedDateTime(2018, 5, 30, 20, 0, 

┌ Info: SELECT "validityintervals"."id" AS "validityintervals_id", "validityintervals"."ref_history" AS "validityintervals_ref_history", "validityintervals"."ref_version" AS "validityintervals_ref_version", "validityintervals"."tsdb_validfrom" AS "validityintervals_tsdb_validfrom", "validityintervals"."tsworld_validfrom" AS "validityintervals_tsworld_validfrom", "validityintervals"."tsdb_invalidfrom" AS "validityintervals_tsdb_invalidfrom", "validityintervals"."tsworld_invalidfrom" AS "validityintervals_tsworld_invalidfrom", "validityintervals"."is_committed" AS "validityintervals_is_committed" FROM "validityintervals" ORDER BY validityintervals.id ASC
└ @ SearchLightPostgreSQL /home/gitpod/.julia/packages/SearchLightPostgreSQL/cIJM3/src/SearchLightPostgreSQL.jl:138


ValidityInterval
| KEY                                | VALUE                         |
|------------------------------------|-------------------------------|
| id::DbId                           | 1                             |
| is_committed::Int64                | 1                             |
| ref_history::DbId                  | 1                             |
| ref_version::DbId                  | 1                             |
| tsdb_invalidfrom::ZonedDateTime    | 2038-01-19T03:14:06.999+00:00 |
| tsdb_validfrom::ZonedDateTime      | 2022-08-25T14:39:25.912+00:00 |
| tsworld_invalidfrom::ZonedDateTime | 2038-01-19T03:14:06.999+00:00 |
| tsworld_validfrom::ZonedDateTime   | 2014-05-30T20:00:01.001+00:00 |
